In [7]:
import pandas as pd
import numpy as np
import json
import os
import uuid
import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers

from tqdm.auto import tqdm
tqdm.pandas()

In [8]:
file_path = r"\\filer\actuarial\Ana\Andrea Request\claimcenternotes\calim_loss_SWR_2023_v2.xlsx"
df = pd.read_excel(file_path)
df.head(1)

,BODY,CREATETIME,SUBJECT,CLAIMID,ID,createdate,CLAIMNUMBER,RPTLSDCC
0,Sent PD estimates with photos to FT. There are...,2023-02-28 14:23:25,FT Review Request- 02,499826,11495896,2023-02-28,C00334246,9466.01


In [9]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')


In [10]:
sentences = df['BODY'].tolist()

In [11]:
queries = [
        "Unlisted driver" 
        "Driver Not Listed"  # Add more queries as needed
]

In [12]:
df['BODY'] = df['BODY'].astype(str)  # Convert everything to string (this will turn NaNs into 'nan' string)


In [13]:
df_filtered = df[df['BODY'].apply(lambda x: not x.startswith('nan') and isinstance(x, str))]


In [14]:
#df_filtered_500 = df_filtered.head(500)

In [15]:
texts = df_filtered['BODY'].tolist()

In [16]:
def encode_corpus(data_frame):
    return model.encode(data_frame['BODY'].tolist(), convert_to_tensor=True)

In [17]:
# Function to process each chunk
def process_chunk(chunk_df):
    corpus_embeddings = encode_corpus(chunk_df)
    
    # Initialize an empty list to collect data
    data = []
    
    # Initialize a dictionary to store top 3 scores for each CLAIMNUMBER
    top3_scores_per_id = {}
    
    # Iterate over each query
    for query in queries:
        # Compute the embedding for the current query
        query_embedding = model.encode(query, convert_to_tensor=True)

        # Compute similarity scores for the current query against all corpus embeddings
        similarity_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]

        # Pair each similarity score with its corresponding CLAIMNUMBER, CLAIMID, SUBJECT, and BODY
        scores_with_ids = list(zip(
            chunk_df['CLAIMNUMBER'].tolist(), 
            chunk_df['CLAIMID'].tolist(), 
            chunk_df['SUBJECT'].tolist(), 
            chunk_df['BODY'].tolist(), 
            similarity_scores.tolist()
        ))

        # Populate the dictionary with lists of (score, CLAIMID, SUBJECT, BODY) tuples, grouped by CLAIMNUMBER
        for claim_number, claim_id, subject, body, score in scores_with_ids:
            if claim_number not in top3_scores_per_id:
                top3_scores_per_id[claim_number] = []
            top3_scores_per_id[claim_number].append((score, claim_id, subject, body))

    # Sort and select top 3 scores for each CLAIMNUMBER
    for claim_number, scores in top3_scores_per_id.items():
        scores.sort(key=lambda x: -x[0])
        top3_scores_per_id[claim_number] = scores[:3]

        # Add to the data list
        for score, claim_id, subject, body in top3_scores_per_id[claim_number]:
            data.append({
                'CLAIMNUMBER': claim_number,
                'CLAIMID': claim_id,
                'SUBJECT': subject,
                'BODY': body,
                'SCORE': score
            })
    
    return pd.DataFrame(data)

# Process the DataFrame in chunks of 500 rows
chunk_size = 500
num_chunks = (len(df) // chunk_size) + 1

all_data = []

for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(df))
    
    print(f"Processing rows {start_idx} to {end_idx}")
    chunk_df = df.iloc[start_idx:end_idx]
    chunk_results = process_chunk(chunk_df)
    all_data.append(chunk_results)

# Concatenate all results into a single DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Save the results to an Excel file
output_file = "scores_report_Ana_final.xlsx"
final_df.to_excel(output_file, index=False, engine='openpyxl')

print("Processing complete. Results saved to", output_file)

Processing rows 0 to 500
Processing rows 500 to 1000
Processing rows 1000 to 1500
Processing rows 1500 to 1506
Processing complete. Results saved to scores_report_Ana_final.xlsx
